In [1]:
import matplotlib.pyplot as plt

from Ballot import Ballot
from DefaultConfigOptions import *
from PartyPrimaryElection import PartyPrimaryElection
from ElectionResult import ElectionResult
from DistrictData import DistrictVotingRecord, DistrictData
from InstantRunoffElection import InstantRunoffElection
from HeadToHeadElection import HeadToHeadElection
from Population import Population
from NDPopulation import NDPopulation
from typing import List, Set, Callable
from Election import Election
from CandidateModel import CandidateModel
import tensorflow as tf
import tensorboard as tb

In [2]:


class ElectionConstructor:
    def __init__(self, constructor: Callable[[List[Ballot], Set[Candidate]], Election], name: str):
        self.constructor = constructor
        self.name = name

    def run(self, ballots: List[Ballot], candidates: Set[Candidate]) -> ElectionResult:
        e = self.constructor(ballots, candidates)
        return e.result()


def construct_irv(ballots: List[Ballot], candidates: Set[Candidate]):
    return InstantRunoffElection(ballots, candidates)


def construct_h2h(ballots: List[Ballot], candidates: Set[Candidate]):
    return HeadToHeadElection(ballots, candidates)

In [3]:
class Sample:
    def __init__(self, opponents: List[Candidate], candidate: Candidate):
        self.opponents = opponents.copy()
        self.candidate = candidate

In [4]:
candidate_model = CandidateModel(input_dim=2, ideology_width=25, ideology_dim=2, n_hidden=50, batch_size=32, learn_rate= 0.001)

class Stats:
    def __init__(self):
        self.model_count = 1e-5
        self.model_winners = 0
        self.random_count = 1e-5
        self.random_winners = 0

    def add_random(self):
        self.random_count += 1

    def add_model(self):
        self.model_count += 1

    def add_random_winner(self):
        self.random_winners += 1

    def add_model_winner(self):
        self.model_winners += 1

    def print(self):
        print("random %6d/%6d %.2f" % (self.random_count, self.random_winners, 100 * self.random_winners / self.random_count), end='')
        print(" model %6d/%6d %.2f" % (self.model_count, self.model_winners, 100 * self.model_winners / self.model_count), end="\r")

stats = Stats()

In [5]:
def run_election(process:  ElectionConstructor):
    pop = NDPopulation(np.array([0, 0]), np.array([40, 40]), Independents)
    voters = pop.generate_voters(1000)

    samples: List[Sample] = []
    candidates: List[Candidate] = []
    is_random = {}
    for i in range(5):
        if np.random.randint(0, 2) == 0 and candidate_model.ready():
            ideology = Ideology(candidate_model.choose_ideology(candidates))
            stats.add_model()
            c = Candidate("c-" + str(i), Independents, ideology, 0)
            is_random[c]  = False
        else:
            ideology = voters[i].ideology
            c = Candidate("c-" + str(i), Independents, ideology, 0)
            is_random[c]  = True
            stats.add_random()

        samples += [Sample(candidates, c)]
        candidates += [c]

    # candidates.append(Candidate("V", Independents, Ideology(np.random.normal(scale=[1.0, 1.0])), quality=0))

    ballots = [Ballot(v, candidates, default_election_config) for v in voters]
    result = process.run(ballots, set(candidates))
    winner = result.winner()

    if is_random[winner]:
        stats.add_random_winner()
    else:
        stats.add_model_winner()

    for s in samples:
        candidate_model.add_sample_from_candidates(s.candidate, s.opponents, result.winner())


In [ ]:
for batch in range(1000 * 1000):

    size_mult = candidate_model.memory.size // 1024
    if size_mult < 2 or random.randrange(0, size_mult) == 0:
        p = ElectionConstructor(construct_irv, "Instant Runoff")
        run_election(p)

    if candidate_model.ready():
        candidate_model.train(32)
        print("memory size %d " % candidate_model.memory.size, end="")
        stats.print()

       concat sample    0.0388/   214 20.04 model     47/     9 19.15
ndom   5192/  1074 20.69 model   4208/   806 19.15memory size 9400 random   5192/  1074 20.69 model   4208/   806 19.15memory size 9405 random   5195/  1075 20.69 model   4210/   806 19.14memory size 9410 random   5197/  1076 20.70 model   4213/   806 19.13memory size 9410 random   5197/  1076 20.70 model   4213/   806 19.13memory size 9410 random   5197/  1076 20.70 model   4213/   806 19.13memory size 9415 random   5198/  1077 20.72 model   4217/   806 19.11memory size 9415 random   5198/  1077 20.72 model   4217/   806 19.11memory size 9415 random   5198/  1077 20.72 model   4217/   806 19.11memory size 9415 random   5198/  1077 20.72 model   4217/   806 19.11memory size 9420 random   5199/  1078 20.73 model   4221/   806 19.10memory size 9430 random   5205/  1078 20.71 model   4225/   808 19.12memory size 9430 random   5205/  1078 20.71 model   4225/   808 19.12memory size 9430 random   5205/  1078 20.71 model   

In [ ]:
candidate_model.timings.print()
